<a href="https://colab.research.google.com/github/khodid/2020_SAI_MONING2/blob/master/RNN_charseq_and_longseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np

In [0]:
sample = "electronic engineering"  # 어느 문자열이든 가능하다.

char_set = list(set(sample))
char_dic = {c : i for i, c in enumerate(char_set)} # ? 

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

#data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

#transform
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)


In [11]:
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    # 실제 모델이 어떻게 예측했는지 관찰하기 위한 코드.
    result = outputs.data.numpy().argmax(axis = 2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction str: ", result_str) 

0 loss:  2.281712770462036 prediction str:  nrnnegirrnenrgrrgngrg
1 loss:  2.091365337371826 prediction str:  nennennenneneenennene
2 loss:  1.9297457933425903 prediction str:  nenteinenneneenennine
3 loss:  1.794550895690918 prediction str:  necteinentengininnini
4 loss:  1.642216444015503 prediction str:  necteinentenginenring
5 loss:  1.528751015663147 prediction str:  ngc renen engengnring
6 loss:  1.4546129703521729 prediction str:  ngc iengn engengnreng
7 loss:  1.3896015882492065 prediction str:  lgc rengn engingnreng
8 loss:  1.3559606075286865 prediction str:  lgctrengn engengnreng
9 loss:  1.3052265644073486 prediction str:  lectrengn engengnreng
10 loss:  1.2671407461166382 prediction str:  lectrengntengengnreng
11 loss:  1.2386850118637085 prediction str:  lectrengntengengereng
12 loss:  1.2103509902954102 prediction str:  lectrengetengengereng
13 loss:  1.1940165758132935 prediction str:  lectrengetengengereng
14 loss:  1.168639898300171 prediction str:  leetrengetengenger

# LongSeq
긴 문장을 고정된 토막으로 잘라서 다루는 방법을 알아보겠다.

In [0]:
# 그냥 LSTM 설명 문서에서 퍼와 봤음
sentence = ("In the above diagram, each line carries an entire vector, "
            "from the output of one node to the inputs of others. "
            "The pink circles represent pointwise operations, "
            "like vector addition, while the yellow boxes are learned neural network layers. ")

In [0]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [0]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [0]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i + sequence_length]
    y_str = sentence[i+1 : i + sequence_length + 1]
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
    
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [0]:
class Net(torch.nn.Module):		# pytorch의 모듈을 '상속' 받는다 ...? 
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)	# 
        x = self.fc(x)
        return x

In [0]:
net = Net(dic_size, hidden_size, 2) # layers에 2를 넣어 두 RNN 레이어를 지나도록 함.

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    # 모델이 예측한 것 해석하기
    results = outputs.argmax(dim=2)
    predict_str = ""
   
    for j, result in enumerate(results):
        # if(i%30==0): print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j==0:
            predict_str += ''.join([char_set[t] for t in result]) 
            # 처음에는 seq_length만큼 가져오고
        else:
            predict_str += char_set[result[-1]]
            # 그 이후로는 업데이트되는 하나씩만 받아오기(맨 마지막 빼고 기존이랑 겹치기 때문)
    print(i, "||", predict_str)

0 || n the ybove ditgram, eecheline carries an entire vector  from the output of one nade to the iaputs of onhers. The pink circles bepresent pointwise operations, like vector addition, whele the yellow boxes are learned neural network layers. 
1 || niahelibore iinrrim,uoanhelinirooatpondoreoueohimoeaaoue aror toiriuooutoofatneuoiueuvonoieuioeopoiaidoihiri, tieuiini  inplorioioniuenaiouniouiririiination  uaowerieaaowetr opooni ooioiuiheunenptuooofet anaraouna.ueaeuptwioiuiore nirers, 
2 || n fi sobofe euamrau  ataheline eartimeeat treiis teaahre trac,fhedoreput on tn, iine fh mi  ireuts bn oiiesvn li daoat ahkkttdenepitdene.oi nthimeuox retinn., aiue feaapret.eipi fe,voeme ii  oeaax eroues baecfnrkiedvteuetneaetiome vires,, 
3 || n th  inove ehtrral, tnchesisesyar tp eanetrchrs fe irr, find nhe intn t pn nn  an e to mhe inests nf nnies   nhe ahae nankgtdene redenestvestiese.on uetiar,,,fine ee akr,tbento ne wh se ah  iekkonenoxe  arc tnrraedwne rnndnp wixs lare v. 
4 || n th  ynove wit